In [1]:
## 生成交叉emb
import pandas as pd
import numpy as np
import gc
from base import Cache
from tqdm import tqdm
from gensim.models import Word2Vec
import sys
from multiprocessing import Pool
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 5000)

# # log
# class Logger(object):
#     def __init__(self, fileN="Default.log"):
#         self.terminal = sys.stdout
#         self.log = open(fileN, "a", encoding='utf-8')

#     def write(self, message):
#         self.terminal.write(message)
#         self.log.write(message)

#     def flush(self):
#         pass
# sys.stdout = Logger("zlh0918log.txt")

def reduce_mem(df, use_float16=False):
    start_mem = df.memory_usage().sum() / 1024**2
    tm_cols = df.select_dtypes('datetime').columns
    for col in df.columns:
        if col in tm_cols:
            continue
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(
                        np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
def gen_list_df(data_):
    data = data_.copy()
#     onehotlist = ['age','city_rank','career','gender']
    feature1,feature2 = list(data.columns)[-2],list(data.columns)[-1]# 主键,emb字段
    print(f'{feature1},{feature2} start!')
    set1 = set(data.query('label<2')[feature1]).intersection(set(data.query('label==2')[feature1]))
    set2 = set(data.query('label<2')[feature2]).intersection(set(data.query('label==2')[feature2]))
    data.loc[~data[feature1].isin(set1),feature1]=-1
    data.loc[~data[feature2].isin(set2),feature2]=-1# unk 词input不在字典里的词都是 -1 每点击过1的词 和没见过的词都用-1替代生成embedding
    
    list_df=data.query('label==1').groupby(feature1)[feature2].apply(lambda x: list(map(str,list(x)))).\
            rename(f'list_{feature2}').reset_index()
    list_df_list = list_df[f'list_{feature2}'].values.tolist()
    print('w2v start!')
    model = Word2Vec(
    list_df_list,
    size=32,
    window=1000,
    workers=5,
    min_count=1,
    sg=0,
    hs=0,
    negative=5,
    iter=5,
    seed=0)
    
    emb_matrix = []
    for seq in list_df_list:
        vec = []
        for w in seq:
            if w in model.wv:
                vec.append(model.wv[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * 32)
            
    emb_matrix = np.array(emb_matrix)
    
    dict_feature1 = {}
    for index,key in enumerate(list(list_df[feature1])):
        dict_feature1[key]=emb_matrix[index,:]# feature1的字典
    dict_feature2 = {}
    for index,key in enumerate(list(model.wv.vocab.keys())):
        dict_feature2[key]=model.wv[key]# feature2的字典
    dict_key_value = {}
    dict_key_value['key']=dict_feature1
    dict_key_value['value']=dict_feature2
    
    Cache.cache_data(dict_key_value, nm_marker=f'EMB_TARGET_DICT_{feature1}__{feature2}_w2v')
    print(emb_matrix.shape,list_df.shape)
    print(f'{feature1},{feature2}done!')

from multiprocessing import Pool
if __name__ == '__main__':
    print('start!')
    data = Cache.reload_cache('CACHE_data_step_1_feature_0917_r5.pkl')
    print(data.shape)
    data['label']=data['label'].fillna(2).astype(int)# mask
    gc.collect()
    data['slot_id_net_type'] = data['slot_id'].values*10+data['net_type'].values# 作为场景交叉项 直接onehot
    # 生成一个emb matrix
    le = LabelEncoder()
    data['slot_id_net_type'] = le.fit_transform(data['slot_id_net_type']).astype(int)
    n_data_matrix= np.eye(data['slot_id_net_type'].nunique())[data['slot_id_net_type'].values]
    n_data_matrix = n_data_matrix.astype(np.int8)
    np.save('./cached_data/EMB_slot_id_net_type_embedding_matrix.npy',n_data_matrix)
    
    # 生成一个emb matrix
    user_fe_list = ['age','city_rank','career','gender','city','device_name','residence','emui_dev']
    item_fe_list = ['task_id','adv_id','adv_prim_id','tags','spread_app_id']
    poc_feature_list = []
    for i,vari in enumerate(user_fe_list):
        for j,varj in enumerate(item_fe_list):
            poc_feature_list.append([vari,varj])
            poc_feature_list.append([varj,vari])
    print(len(poc_feature_list))
    datalist = []
    for item in poc_feature_list:
        datalist.append(data[['uid','index','pt_d','label',item[0],item[1]]])# 拆分数据集
    with Pool(4) as p:
        result = p.map(gen_list_df, datalist)


start!


[2020-09-19 02:09:43] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_data_step_1_feature_0917_r5.pkl


(8601298, 63)
80
age,task_id start!
adv_prim_id,age start!
city_rank,task_id start!
adv_prim_id,city_rank start!
w2v start!


[2020-09-19 02:14:40] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-19 02:14:40] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-19 02:14:41] - word2vec.py[line:1407] - INFO: collected 1715 word types from a corpus of 1266883 raw words and 7 sentences
[2020-09-19 02:14:41] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-19 02:14:41] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 1715 unique words (100% of original 1715, drops 0)
[2020-09-19 02:14:41] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 1266883 word corpus (100% of original 1266883, drops 0)
[2020-09-19 02:14:41] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 1715 items
[2020-09-19 02:14:41] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 76 most-common words
[2020-09-19 02:14:41] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 752512

w2v start!


...

[2020-09-19 03:41:18] - base_any2vec.py[line:1291] - INFO: EPOCH 4 - PROGRESS: at 58.33% examples, 22869 words/s, in_qsize 9, out_qsize 0
[2020-09-19 03:41:19] - base_any2vec.py[line:1291] - INFO: EPOCH 4 - PROGRESS: at 72.22% examples, 24752 words/s, in_qsize 9, out_qsize 0
[2020-09-19 03:41:20] - base_any2vec.py[line:1291] - INFO: EPOCH 4 - PROGRESS: at 86.11% examples, 25487 words/s, in_qsize 5, out_qsize 0
[2020-09-19 03:41:21] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 4 more threads
[2020-09-19 03:41:21] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 3 more threads
[2020-09-19 03:41:21] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 2 more threads
[2020-09-19 03:41:21] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 1 more threads
[2020-09-19 03:41:22] - base_any2vec.py[line:1291] - INFO: EPOCH 4 - PROGRESS: at 100.00% examples, 28204 words/s, in_qsiz

(36, 32) (36, 2)
residence,adv_prim_iddone!
w2v start!


[2020-09-19 03:42:48] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-19 03:42:48] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-19 03:42:49] - word2vec.py[line:1407] - INFO: collected 17 word types from a corpus of 1266883 raw words and 50 sentences
[2020-09-19 03:42:49] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-19 03:42:49] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 17 unique words (100% of original 17, drops 0)
[2020-09-19 03:42:49] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 1266883 word corpus (100% of original 1266883, drops 0)
[2020-09-19 03:42:49] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 17 items
[2020-09-19 03:42:49] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 10 most-common words
[2020-09-19 03:42:49] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 120087 word c

[2020-09-19 03:43:18] - base_any2vec.py[line:1332] - INFO: EPOCH - 5 : training on 1266883 raw words (109455 effective words) took 5.9s, 18473 effective words/s
[2020-09-19 03:43:18] - base_any2vec.py[line:1368] - INFO: training on a 6334415 raw words (546436 effective words) took 28.9s, 18911 effective words/s
[2020-09-19 03:43:20] - __init__.py[line:111] - INFO: Cache Successfully! File name: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_EMB_TARGET_DICT_spread_app_id__emui_dev_w2v.pkl


(50, 32) (50, 2)
spread_app_id,emui_devdone!
